In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input
import os
import pandas as pd
import numpy as np


In [5]:
import tensorflow as tf
print(tf.__version__)



2.19.0


In [2]:
TRAIN_DIR = "images/train"
TEST_DIR = "images/test"

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels


In [4]:
train= pd.DataFrame()
train['image'],train['label']=createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
test= pd.DataFrame()
test['image'],test['label']=createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [6]:
import sys
!{sys.executable} -m pip install Pillow

In [7]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tqdm import tqdm
import cv2

In [8]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale', target_size=(48, 48))
        img = img_to_array(img).astype("uint8")

        # Apply CLAHE
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img = clahe.apply(img.squeeze())
        img = np.expand_dims(img, axis=-1)  # shape = (48, 48, 1)
        features.append(img)

    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [9]:
train_features = extract_features(train['image'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28821/28821 [02:47<00:00, 172.50it/s]


In [10]:
test_features = extract_features(test['image'])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7066/7066 [00:40<00:00, 172.77it/s]


In [11]:
x_train = train_features/255.0
x_test = test_features/255.0

In [12]:
import sys
!{sys.executable} -m pip install scikit-learn

In [13]:
from sklearn.preprocessing import LabelEncoder 

In [14]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [15]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])

In [16]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [34]:
model = Sequential()
model.add(Input(shape=(48, 48, 1)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.3))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


In [35]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)
datagen.fit(x_train)

history = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    validation_data=(x_test, y_test),
    epochs=150,
    callbacks=[earlystop, lr_reduce]
)


Epoch 1/150
378/451 ━━━━━━━━━━━━━━━━━━━━ 9s 123ms/step - accuracy: 0.2247 - loss: 1.8417

In [ ]:
model_json=model.to_json()
with open("emotiondetector1.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector1.keras")

In [ ]:
from keras.models import model_from_json
from tensorflow.keras.preprocessing.image import load_img


In [ ]:
json_file=open("emotiondetector1.json","r")
model_json=json_file.read()
json_file.close()
model=model_from_json(model_json)
model.load_weights("emotiondetector1.keras")

In [ ]:
label=["angry","disgust","fear","happy","neutral","sad","surprise"]

In [ ]:
def ef(image):
    img = load_img(image, color_mode='grayscale', target_size=(48, 48))
    feature= np.array(img)
    feature=feature.reshape(1,48,48,1)
    return feature/255.0

In [ ]:
image="images/train/sad/86.jpg"
print("OG image is angry")
img=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("Model prediction:- ",pred_label)

In [ ]:
!pip install matplotlib


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
image="images/train/sad/86.jpg"
print("OG image is angry")
img=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("Model prediction:- ",pred_label)
plt.imshow(img.reshape(48,48),cmap="gray")

In [ ]:
image="images/train/fear/2.jpg"
print("OG image is angry")
img=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("Model prediction:- ",pred_label)
plt.imshow(img.reshape(48,48),cmap="gray")

In [ ]:
image="images/train/disgust/299.jpg"
print("OG image is angry")
img=ef(image)
pred=model.predict(img)
pred_label=label[pred.argmax()]
print("Model prediction:- ",pred_label)
plt.imshow(img.reshape(48,48),cmap="gray")